In [59]:
import pandas as pd

receipts = pd.read_csv("receipts.csv")
payments = pd.read_csv("payments.csv")
torque = pd.read_csv("torque.csv")
discounts = pd.read_csv("discounts.csv")
items = pd.read_csv("items.csv")


In [107]:
soma_torque_por_loja = torque.groupby('shop_id')['net_torque'].sum().reset_index()
soma_torque_por_loja.columns = ['shop_id', 'soma_net_torque']


soma_torque_por_loja['soma_net_torque_normalizado'] = (
 (soma_torque_por_loja['soma_net_torque'] - soma_torque_por_loja['soma_net_torque'].min()) /
    (soma_torque_por_loja['soma_net_torque'].max() - soma_torque_por_loja['soma_net_torque'].min())
)

print(soma_torque_por_loja)


                        shop_id  soma_net_torque  soma_net_torque_normalizado
0       highway-adidas-shopping     25923.459287                     0.000000
1  highway-avenida-nova-balanca     30197.736906                     1.000000
2            highway-praca-ekin     26594.364142                     0.156963
3     highway-rua-bens-perdidos     28365.698066                     0.571380


In [68]:
df_filtrado = receipts[receipts['canceled'] == False]

# Agrupa por loja e soma o total_value
faturamento_por_loja = df_filtrado.groupby('shop_id')['total_value'].sum().reset_index()

# Renomeia a coluna para facilitar a leitura
faturamento_por_loja.columns = ['shop_id', 'faturamento']
faturamento_por_loja['faturamento_normalizado'] = (
    (faturamento_por_loja['faturamento'] - faturamento_por_loja['faturamento'].min()) / (faturamento_por_loja['faturamento'].max() - faturamento_por_loja['faturamento'].min())
)

faturamento_por_loja

,shop_id,faturamento,faturamento_normalizado
0,highway-adidas-shopping,1937390.99,1.000000
1,highway-avenida-nova-balanca,1424611.80,0.000000
2,highway-praca-ekin,1523550.32,0.192946
3,highway-rua-bens-perdidos,1829981.55,0.790535


In [74]:
# Filtra apenas os pedidos não cancelados
df_filtrado = receipts[receipts['canceled'] == False]

# Agrupa por loja: soma total_value e conta número de pedidos
agrupado = df_filtrado.groupby('shop_id').agg(
    faturamento=('total_value', 'sum'),
    num_pedidos=('total_value', 'count')
).reset_index()

# Calcula o ticket médio
agrupado['ticket_medio'] = agrupado['faturamento'] / agrupado['num_pedidos']
# Normaliza o ticket médio
agrupado['ticket_medio_normalizado'] = (
    (agrupado['ticket_medio'] - agrupado['ticket_medio'].min()) /
    (agrupado['ticket_medio'].max() - agrupado['ticket_medio'].min())
)


print(agrupado[['shop_id', 'ticket_medio', 'ticket_medio_normalizado']])


                        shop_id  ticket_medio  ticket_medio_normalizado
0       highway-adidas-shopping     49.289955                  1.000000
1  highway-avenida-nova-balanca     40.739278                  0.000000
2            highway-praca-ekin     42.502659                  0.206227
3     highway-rua-bens-perdidos     42.156731                  0.165771


In [90]:
# Filtra apenas os itens não cancelados
df_itens_filtrado = discounts[discounts['canceled'] == False]

# Junta com a tabela de recibos para pegar o shop_id
df_merged = df_itens_filtrado.merge(
    receipts[['identifier', 'shop_id']],
    left_on='fk_receipt_identifier',
    right_on='identifier',
    how='left'
)

# Agrupa por loja e soma o total de descontos
descontos_por_loja = df_merged.groupby('shop_id')['total'].sum().reset_index()

descontos_por_loja.columns = ['shop_id', 'total_descontos']
descontos_por_loja['descontos_normalizados'] = (
    (descontos_por_loja['total_descontos'] - descontos_por_loja['total_descontos'].min()) /
    (descontos_por_loja['total_descontos'].max() - descontos_por_loja['total_descontos'].min())
)
descontos_por_loja['descontos_invertidos'] = 1 - descontos_por_loja['descontos_normalizados']


descontos_por_loja.head()


,shop_id,total_descontos,descontos_normalizados,descontos_invertidos
0,highway-adidas-shopping,23161.36,0.028592,0.971408
1,highway-avenida-nova-balanca,22708.88,0.000000,1.000000
2,highway-praca-ekin,25066.50,0.148978,0.851022
3,highway-rua-bens-perdidos,38534.14,1.000000,0.000000


In [114]:
dados_dash = pd.DataFrame({
    'shop_id': [
        'highway-adidas-shopping',
        'highway-avenida-nova-balanca',
        'highway-praca-ekin',
        'highway-rua-bens-perdidos' ],

    'descontos_invertidos': [0.971408, 1, 0.851022, 0 ],
    'ticket_medio_normalizado': [1, 0, 0.206227, 0.165771],
    'faturamento_normalizado': [1.0, 0, 0.192946, 0.790535],
    'soma_net_torque_normalizado': [0, 1.0, 0.156963, 0.571380],
})

dados_dash

dados_dash["Metrica"] = dados_dash['descontos_invertidos'] + dados_dash['ticket_medio_normalizado'] + dados_dash['faturamento_normalizado'] + dados_dash['soma_net_torque_normalizado']


dados_dash.to_csv('dados_dash1.csv', index=False, sep=';', decimal=',')

In [ ]:
dados_dash =
DATATABLE (
    "shop_id", STRING,
    "descontos_invertidos", DOUBLE,
    "ticket_medio_normalizado", DOUBLE,
    "faturamento_normalizado", DOUBLE,
    "soma_net_torque_normalizado", DOUBLE,
    {
        { "highway-adidas-shopping", 0.971408, 1.0, 1.0, 0.0 },
        { "highway-avenida-nova-balanca", 1.0, 0.0, 0.0, 1.0 },
        { "highway-praca-ekin", 0.851022, 0.206227, 0.192946, 0.156963 },
        { "highway-rua-bens-perdidos", 0.0, 0.165771, 0.790535, 0.571380 }
    }
)
